# Workbook to scrape player career information

**Purpose of workbook:** Scrape player career information such as total games, goals, club points per game, minutes per goal.  Also scrape this at the season level.

## Import libraries and connect to local SQLite

In [1]:
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
import pandas as pd
from pandas.io import sql
import sqlite3
import numpy as np
connection = sqlite3.connect('../sql/career_stats.db.sqlite')
connection.text_factory = str

import time

### Create functions to scrape player information

In [131]:
def convert_to_soup(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36"})
    soup = BeautifulSoup(r.content,"html.parser")
    
    return soup

def info_from_url(url):
    # Extract player id from url
    pid = re.search('spieler/(.*)/saison//verein', url).group(1)

    # Extract player name from url
    pname = re.search('.com/(.*)/leistungsdatendetails', url).group(1)

    return pid, pname
    
def extract_intl(soup):
    # Extract international caps and goals from page
    #int1 = soup.find_all('span', class_="dataValue")[8].text.split("/") # Intl info is 9th dataValue item
    
    try:
        int_split_country = str(soup).split("nternational:")
        int_soup_country = BeautifulSoup(int_split_country[1], "html.parser")
        int_country_text = int_soup_country.find('a', class_="vereinprofil_tooltip").get_text()
        int_country_id = int_soup_country.find('a', class_="vereinprofil_tooltip")['id']
        
        int_split = str(soup).split("International caps/goals:")
        int_soup = BeautifulSoup(int_split[1], "html.parser")
        int_text = int_soup.find('span', class_="dataValue").get_text().split("/")
        intl_caps = float(int_text[0])
        intl_goals = float(int_text[1])
    
    except:
        intl_caps = np.NaN
        intl_goals = np.NaN
        int_country_id = np.NaN
        int_country_text = np.NaN
    
    
    return intl_caps, intl_goals, int_country_id, int_country_text

def club_ids(soup, temp_df):
    # Extract club IDs
    club_id_count = temp_df.shape[0] # Calculate number of club IDs to find
    
    clubs = soup.find_all('a', class_="vereinprofil_tooltip")
    list_of_clubs = []
    index_start = len(clubs) - club_id_count
    for x in clubs[index_start:]:
        list_of_clubs.append(int(x['id']))
        
    return list_of_clubs

def league_ids(soup):
    # Extract league IDs
    leagues = soup.find_all('td', class_="hauptlink no-border-links")
    list_of_leagues = []
    
    if len(leagues) > 0:    
        for x in leagues:
            list_of_leagues.append(x.get_text())

    else: # For pages where no stats are available, I have inserted two empty rows - but I only need one since
            # summary row gets removed before this function is called
        list_of_leagues = [np.NaN]
        
    return list_of_leagues
    
def player_table(soup):
    # Isolate and convert player table
    tables = soup.find_all('table')
    
    if len(tables) > 1: # Check if a table of detailed stats exists
        table = tables[1]
        player_df = pd.read_html(str(table))[0]
    
        check_if_outfield = soup.find_all('span', title="Penalty goals")

        if len(check_if_outfield) > 0:

            # Name the columns in the table (necessary since the naming is not accurate on direct conversion)
            player_df.columns = ['season','comp_gap','comp','club','squad','apps','PPM','goals','assists','own_goals',
                                 'sub_on','sub_off','yellow','yel_red','red','pen_goals','MPG','minutes']

            # Drop columns not needed for upload
            player_df = player_df.drop(['comp_gap','club'], axis = 1)

            # Add missing columns
            player_df['conceded_goals'] = np.NaN # since player not a keeper
            player_df['clean_sheets'] = np.NaN # since player not a keeper

        else:

            # Name the columns in the table (necessary since the naming is not accurate on direct conversion)
            player_df.columns = ['season','comp_gap','comp','club','squad','apps','PPM','goals','own_goals',
                                 'sub_on','sub_off','yellow','yel_red','red','conceded_goals','clean_sheets',
                                 'minutes']

            # Drop columns not needed for upload
            player_df = player_df.drop(['comp_gap','club'], axis = 1)

            # Add missing columns
            player_df['assists'] = np.NaN # since player not an outfield player
            player_df['pen_goals'] = np.NaN # since player not an outfield player
            player_df['MPG'] = np.NaN # since player not an outfield player


        return player_df

    else: # If a table of detailed stats doesn't exist
        player_df = pd.DataFrame(columns = ['season','comp_gap','comp','club','squad','apps','PPM','goals',
                                            'own_goals','sub_on','sub_off','yellow','yel_red','red',
                                            'conceded_goals','clean_sheets','minutes','assists','pen_goals',
                                            'MPG'])
        
        # Drop columns not needed for upload
        player_df = player_df.drop(['comp_gap','club'], axis = 1)
        
        player_df.loc[0] = np.NaN
        player_df.loc[1] = np.NaN
        
        return player_df
    
def minutesremove(value):
    # Code to remove minute sign and convert to float
    val = str(value)
    val = val.replace("'","").replace(".","")
    return val

In [106]:
def create_dfs(url):
    
    soup = convert_to_soup(url) # Convert URL to soup
    
    pid, pname = info_from_url(url) # Extract player ID and player name
    
    intl_caps, intl_goals, int_country_id, int_country_text = extract_intl(soup) # Extract international info
    
    temp_df = player_table(soup) # Create player table
    
    temp_df['PPM'] = temp_df['PPM'].apply(pd.to_numeric, errors = 'coerce')
    temp_df['PPM'] = temp_df['PPM']/100 # Fix points per match
    
    # Clean the minutes and MPG columns
    
    temp_df['minutes'] = temp_df['minutes'].apply(minutesremove)
    temp_df['MPG'] = temp_df['MPG'].apply(minutesremove)
    temp_df['minutes'] = temp_df['minutes'].apply(pd.to_numeric, errors = 'coerce')
    temp_df['MPG'] = temp_df['MPG'].apply(pd.to_numeric, errors = 'coerce')
    
    # Add columns to df

    temp_df['pid'] = int(pid)
    temp_df['pname'] = pname
    
    # Extract final row and place in new DataFrame (since this is the total player career information) 
    # & add intl stats

    temp_summary = temp_df.iloc[-1:,2:]
    temp_summary['intl_caps'] = intl_caps
    temp_summary['intl_goals'] = intl_goals
    temp_summary['intl_country'] = int_country_text
    temp_summary['intl_country_id'] = int_country_id
    
    temp_df = temp_df.iloc[:-1,:] # Remove final row from detailed summary & add list of clubs
    
    list_of_clubs = club_ids(soup, temp_df) # Create list of club ids
    
    list_of_leagues = league_ids(soup) # Create list of league ids
    
    # Add club ids and league names

    temp_df['club_id'] = list_of_clubs
    temp_df['league_name'] = list_of_leagues

    # Code to convert columns to float and remove errors

    cols = temp_df.columns[2:14]
    temp_df[cols] = temp_df[cols].apply(pd.to_numeric, errors='coerce', axis=1)

    cols_summary = temp_summary.columns[0:12]
    temp_summary[cols] = temp_summary[cols_summary].apply(pd.to_numeric, errors='coerce', axis=1)

    # Upload dataframes into SQL

    #temp_summary.to_sql(name = 'summary_stats', con = connection, if_exists = 'replace', index = False)
    #temp_df.to_sql(name = 'detailed_stats', con = connection, if_exists = 'replace', index = False)
    
    return temp_summary, temp_df

### Initiate empty dataframes for each scrape

In [ ]:
# Initiate dataframes to store each player's stats and summary stats details (inc International caps & goals):

total_stats_A = pd.DataFrame(columns=['season','comp','squad','apps','PPM','goals','assists','own_goals','sub_on',
                                    'sub_off','yellow','yel_red','red','pen_goals','MPG','conceded_goals',
                                    'clean_sheets','minutes','pid','pname','club_id','league_name'])

summary_stats_A = pd.DataFrame(columns=['squad','apps','PPM','goals','assists','own_goals','sub_on',
                                      'sub_off','yellow','yel_red','red','pen_goals','MPG','conceded_goals',
                                      'clean_sheets','minutes','pid','pname','intl_caps','intl_goals',
                                      'intl_country','intl_country_id'])

In [56]:
# Initiate dataframes to store each player's stats and summary stats details (inc International caps & goals):

total_stats_B = pd.DataFrame(columns=['season','comp','squad','apps','PPM','goals','assists','own_goals','sub_on',
                                    'sub_off','yellow','yel_red','red','pen_goals','MPG','conceded_goals',
                                    'clean_sheets','minutes','pid','pname','club_id','league_name'])

summary_stats_B = pd.DataFrame(columns=['squad','apps','PPM','goals','assists','own_goals','sub_on',
                                      'sub_off','yellow','yel_red','red','pen_goals','MPG','conceded_goals',
                                      'clean_sheets','minutes','pid','pname','intl_caps','intl_goals',
                                      'intl_country','intl_country_id'])

In [99]:
# Initiate dataframes to store each player's stats and summary stats details (inc International caps & goals):

total_stats_C = pd.DataFrame(columns=['season','comp','squad','apps','PPM','goals','assists','own_goals','sub_on',
                                    'sub_off','yellow','yel_red','red','pen_goals','MPG','conceded_goals',
                                    'clean_sheets','minutes','pid','pname','club_id','league_name'])

summary_stats_C = pd.DataFrame(columns=['squad','apps','PPM','goals','assists','own_goals','sub_on',
                                      'sub_off','yellow','yel_red','red','pen_goals','MPG','conceded_goals',
                                      'clean_sheets','minutes','pid','pname','intl_caps','intl_goals',
                                      'intl_country','intl_country_id'])

### Connect to player market value SQL database to extract player info to create list of player page URLs

In [5]:
tx_sqlite_db = '../sql/stats.db.sqlite'
tx_conn = sqlite3.connect(tx_sqlite_db) 
tx = tx_conn.cursor()

In [6]:
# This is my SQL query to select everything:
query_vals = 'select * from market_vals'

# Use `read_sql` from the `pandas` SQL library and set it equal to a DF object:
vals_df = pd.read_sql(query_vals, con = tx_conn)

In [7]:
# Extract player name and player ID

player_df = vals_df[['name','pid']].copy()

In [8]:
# Code to format name into url string

player_df['url_name'] = player_df['name'].map(lambda x: x.lower().replace("'","").replace(" ","-"))

# Drop duplicates

player_df = player_df.drop_duplicates()

In [9]:
# Create list of Player URLs to scrape

list_of_player_urls = []

for index, row in player_df.iterrows():
    part_A = "https://www.transfermarkt.com/" + row['url_name']
    part_B = "/leistungsdatendetails/spieler/" + str(row['pid']).replace(".0","")
    part_C = "/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
    url_address = part_A + part_B + part_C
    list_of_player_urls.append(url_address)

In [10]:
len(list_of_player_urls)

8229

### Scrape player URLs

In [13]:
# Code to scrape each URL

start_time=time.time()

count = 0
failed_urls = []


for url in list_of_player_urls:
    count += 1
    try:
        temp_summary, temp_df = create_dfs(url)
        total_stats = pd.concat([total_stats, temp_df])
        summary_stats = pd.concat([summary_stats, temp_summary])
    except:
        print(url)
        failed_urls.append(url)
    
    if count % 100 == 0:
        end_time=time.time()
        seconds_elapsed = start_time - end_time
        seconds_per_url = float(seconds_elapsed)/count
        urls_remaining = len(list_of_player_urls) - count
        time_remaining = seconds_per_url * urls_remaining
        print("URLs loaded: {}. Time remaining: {}".format(count, time_remaining))


https://www.transfermarkt.com/callum-richardson/leistungsdatendetails/spieler/400531/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
URLs loaded: 100. Time remaining: -24827.3542721
https://www.transfermarkt.com/joshua-barresi/leistungsdatendetails/spieler/261616/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
URLs loaded: 200. Time remaining: -25176.6263848
https://www.transfermarkt.com/jordan-pudler/leistungsdatendetails/spieler/420211/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
URLs loaded: 300. Time remaining: -24686.9916642
https://www.transfermarkt.com/bai-antoniou/leistungsdatendetails/spieler/467020/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/pierce-waring/leistungsdatendetails/spieler/400000/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/emilio-martinez/leistungsdatendetails/spieler/467023/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_i

https://www.transfermarkt.com/dieumerci-mbokani/leistungsdatendetails/spieler/44624/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/ricky-van-wolfswinkel/leistungsdatendetails/spieler/41355/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/gary-hooper/leistungsdatendetails/spieler/42513/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/lewis-grabban/leistungsdatendetails/spieler/35413/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/sylvain-distin/leistungsdatendetails/spieler/3356/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/stéphane-zubar/leistungsdatendetails/spieler/56829/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/baily-cargill/leistungsdatendetails/spieler/293450/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/p

https://www.transfermarkt.com/lucas-melano/leistungsdatendetails/spieler/220753/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/darlington-nagbe/leistungsdatendetails/spieler/173489/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
URLs loaded: 2000. Time remaining: -23874.0333019
URLs loaded: 2100. Time remaining: -23458.9259208
https://www.transfermarkt.com/t.j.-casner/leistungsdatendetails/spieler/424209/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
URLs loaded: 2200. Time remaining: -22978.8691093
https://www.transfermarkt.com/pc/leistungsdatendetails/spieler/292221/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/bryan-reynolds/leistungsdatendetails/spieler/475208/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
URLs loaded: 2300. Time remaining: -22473.8999429
https://www.transfermarkt.com/gianluca-busio/leistungsdatendetails/spieler/504209/saison//verei

https://www.transfermarkt.com/dario-van-den-buijs/leistungsdatendetails/spieler/235274/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/sebastian-jakubiak/leistungsdatendetails/spieler/124084/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/niels-leemhuis/leistungsdatendetails/spieler/450201/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/zeki-erkilinc/leistungsdatendetails/spieler/387218/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/nicolás-freire/leistungsdatendetails/spieler/266809/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/timon-wellenreuther/leistungsdatendetails/spieler/196829/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/dylan-ryan/leistungsdatendetails/spieler/480022/saison//verein/0/liga/0/wettbewerb//pos/0/tra

https://www.transfermarkt.com/ángel-di-maría/leistungsdatendetails/spieler/45320/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/lucas/leistungsdatendetails/spieler/77100/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/edinson-cavani/leistungsdatendetails/spieler/48280/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/odsonne-edouard/leistungsdatendetails/spieler/344152/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/jean-kévin-augustin/leistungsdatendetails/spieler/264956/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/anthony-lopes/leistungsdatendetails/spieler/60561/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/mathieu-gorgelin/leistungsdatendetails/spieler/60543/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1

...

https://www.transfermarkt.com/marcel-risse/leistungsdatendetails/spieler/51176/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/yuya-osako/leistungsdatendetails/spieler/108650/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/anthony-modeste/leistungsdatendetails/spieler/50512/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/simon-zoller/leistungsdatendetails/spieler/79967/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/philipp-hosiner/leistungsdatendetails/spieler/37788/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/bard-finne/leistungsdatendetails/spieler/202925/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/örjan-nyland/leistungsdatendetails/spieler/73517/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://w

https://www.transfermarkt.com/nils-rütten/leistungsdatendetails/spieler/188010/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/lászló-bénes/leistungsdatendetails/spieler/243188/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/mike-feigenspan/leistungsdatendetails/spieler/315256/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/ba-muaka-simakala/leistungsdatendetails/spieler/227974/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/julio-villalba/leistungsdatendetails/spieler/401717/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/oliver-oschkenat/leistungsdatendetails/spieler/124352/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/douglas-santos/leistungsdatendetails/spieler/220793/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/

https://www.transfermarkt.com/marc-torrejón/leistungsdatendetails/spieler/38266/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/caglar-söyüncü/leistungsdatendetails/spieler/320141/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/manuel-gulde/leistungsdatendetails/spieler/45683/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/jonas-föhrenbach/leistungsdatendetails/spieler/192169/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/christian-günter/leistungsdatendetails/spieler/93707/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/fabian-menig/leistungsdatendetails/spieler/124507/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/lukas-kübler/leistungsdatendetails/spieler/93604/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1

https://www.transfermarkt.com/robin-koch/leistungsdatendetails/spieler/328784/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/yoric-ravet/leistungsdatendetails/spieler/82461/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
URLs loaded: 8200. Time remaining: -56.8769225526
https://www.transfermarkt.com/mohamed-dräger/leistungsdatendetails/spieler/192167/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/julian-pollersbeck/leistungsdatendetails/spieler/228975/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/bjarne-thoelke/leistungsdatendetails/spieler/94210/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/jonas-behounek/leistungsdatendetails/spieler/270159/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/fiete-arp/leistungsdatendetails/spieler/343337/saison//verein

UnicodeEncodeError: 'ascii' codec can't encode character u'\xe9' in position 1: ordinal not in range(128)

In [53]:
# Save failed URL list to CSV

failed_url_df = pd.DataFrame(failed_urls)

# Create new database & connection to store failed URLs

failed_sqlite_db = '../sql/failed_urls.db.sqlite'
fail_con = sqlite3.connect(failed_sqlite_db) 
fail = fail_con.cursor()

# Upload dataframes to SQL database

failed_url_df.to_sql(name = 'failed_urls', con = fail_con, if_exists = 'replace', index = False)

### Secondary scrape of failed URLs (after updating code for failed items)

In [57]:
# Code to scrape URLs and save to CSV

start_time=time.time()

count = 0
failed_urls_B = []


for url in failed_urls:
    count += 1
    try:
        temp_summary, temp_df = create_dfs(url)
        total_stats_B = pd.concat([total_stats_B, temp_df])
        summary_stats_B = pd.concat([summary_stats_B, temp_summary])
    except:
        print(url)
        failed_urls_B.append(url)
    
    if count % 100 == 0:
        end_time=time.time()
        seconds_elapsed = end_time - start_time
        seconds_per_url = float(seconds_elapsed)/count
        urls_remaining = len(failed_urls) - count
        time_remaining = seconds_per_url * urls_remaining
        print("URLs loaded: {}. Time remaining: {}".format(count, time_remaining))




https://www.transfermarkt.com/callum-richardson/leistungsdatendetails/spieler/400531/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/joshua-barresi/leistungsdatendetails/spieler/261616/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/jordan-pudler/leistungsdatendetails/spieler/420211/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/bai-antoniou/leistungsdatendetails/spieler/467020/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/pierce-waring/leistungsdatendetails/spieler/400000/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/emilio-martinez/leistungsdatendetails/spieler/467023/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/mark-moric/leistungsdatendetails/spieler/496653/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plu

URLs loaded: 3700. Time remaining: 1817.9112905
URLs loaded: 3800. Time remaining: 1410.52503818
URLs loaded: 3900. Time remaining: 999.898446871
URLs loaded: 4000. Time remaining: 587.369325478
URLs loaded: 4100. Time remaining: 176.410763481


UnicodeEncodeError: 'ascii' codec can't encode character u'\xe1' in position 0: ordinal not in range(128)

In [61]:
# Save failed URL list to CSV

failed_url_B_df = pd.DataFrame(failed_urls_B)

# Create new database & connection to store failed URLs

failed_sqlite_db = '../sql/failed_urls.db.sqlite'
fail_con = sqlite3.connect(failed_sqlite_db) 
fail = fail_con.cursor()

# Upload dataframes to SQL database

failed_url_B_df.to_sql(name = 'failed_urls_B', con = fail_con, if_exists = 'replace', index = False)

### Third scrape of URLs (after updating code for failed items)

In [119]:
# Code to scrape URLs and save to CSV

start_time=time.time()

count = 0
failed_urls_C = []


for url in failed_url_list_3:
    count += 1
    try:
        temp_summary, temp_df = create_dfs(url)
        total_stats_C = pd.concat([total_stats_C, temp_df])
        summary_stats_C = pd.concat([summary_stats_C, temp_summary])
    except:
        print(url)
        failed_urls_C.append(url)
    
    if count % 10 == 0:
        end_time=time.time()
        seconds_elapsed = end_time - start_time
        seconds_per_url = float(seconds_elapsed)/count
        urls_remaining = len(failed_urls) - count
        time_remaining = seconds_per_url * urls_remaining
        print("URLs loaded: {}. Time remaining: {}".format(count, time_remaining))

URLs loaded: 10. Time remaining: 9221.06391921
URLs loaded: 20. Time remaining: 10756.2186682
https://www.transfermarkt.com/pc/leistungsdatendetails/spieler/292221/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/frantisek-rajtoral-†/leistungsdatendetails/spieler/34527/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
URLs loaded: 30. Time remaining: 10953.4003332
URLs loaded: 40. Time remaining: 11104.5222755
https://www.transfermarkt.com/mathí­as-olivera/leistungsdatendetails/spieler/376514/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
https://www.transfermarkt.com/niklas-feierabend-†/leistungsdatendetails/spieler/297202/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1


### Save dataframe into SQL

In [22]:
# Create new database & connection to store career stats

career_sqlite_db = '../sql/career_stats.db.sqlite'
career_con = sqlite3.connect(career_sqlite_db) 
career = career_con.cursor()

# Upload dataframes to SQL database

summary_stats_B.to_sql(name = 'summary_stats', con = career_con, if_exists = 'replace', index = False)
total_stats_B.to_sql(name = 'detailed_stats', con = career_con, if_exists = 'replace', index = False)

### Save 2nd scrape into a new SQL

In [58]:
# Create new database & connection to store career stats

career_sqlite_B_db = '../sql/career_stats_B.db.sqlite'
career_con_B = sqlite3.connect(career_sqlite_B_db) 
career_B = career_con_B.cursor()

# Upload dataframes to SQL database

summary_stats_B.to_sql(name = 'summary_stats_B', con = career_con_B, if_exists = 'replace', index = False)
total_stats_B.to_sql(name = 'detailed_stats_B', con = career_con_B, if_exists = 'replace', index = False)

### Save 3rd scrape into a new SQL

In [122]:
# Create new database & connection to store career stats

career_sqlite_C_db = '../sql/career_stats_C.db.sqlite'
career_con_C = sqlite3.connect(career_sqlite_C_db) 
career_C = career_con_C.cursor()

# Upload dataframes to SQL database

summary_stats_C.to_sql(name = 'summary_stats_C', con = career_con_C, if_exists = 'replace', index = False)
total_stats_C.to_sql(name = 'detailed_stats_C', con = career_con_C, if_exists = 'replace', index = False)

### Clean failed URLs (after 2nd scrape)

In [116]:
# Remove "." in URL which appear after .com

failed_url_list_3 = []

for x in failed_urls_B:
    split_url = x.split(".com")
    remove_dot = split_url[1].replace(".","")
    new_url = split_url[0] + ".com" + remove_dot
    failed_url_list_3.append(new_url)

In [117]:
len(failed_url_list_3)

45

### Test failed URLs

In [32]:
url_test = "https://www.transfermarkt.com/bai-antoniou/leistungsdatendetails/spieler/467020/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
r = requests.get(url_test, headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36"})
soup = BeautifulSoup(r.content,"html.parser")

In [33]:
tables = soup.find_all('table')

In [64]:
url_test_3 = failed_urls_B[0]
url_test_3

u'https://www.transfermarkt.com/callum-richardson/leistungsdatendetails/spieler/400531/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1'

In [101]:
url = failed_urls_B[0]

In [102]:
soup = convert_to_soup(url) # Convert URL to soup
    
pid, pname = info_from_url(url) # Extract player ID and player name

intl_caps, intl_goals, int_country_id, int_country_text = extract_intl(soup) # Extract international info

temp_df = player_table(soup) # Create player table

In [103]:
temp_df

,season,comp_gap,comp,club,squad,apps,PPM,goals,own_goals,sub_on,sub_off,yellow,yel_red,red,conceded_goals,clean_sheets,minutes,assists,pen_goals,MPG
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Check shape of each of the dataframes (to ensure consistency before appending to each other)

In [144]:
print(summary_stats.shape)
print(summary_stats_B.shape)
print(summary_stats_C.shape)

(4086, 22)
(4098, 22)
(41, 22)


In [145]:
print(total_stats.shape)
print(total_stats_B.shape)
print(total_stats_C.shape)

(86922, 22)
(80460, 22)
(225, 22)


### Consolidate dataframes and store in SQLite

In [ ]:
# Drop columns not needed for upload
#total_stats_C = total_stats_C.drop(['comp_gap','club'], axis = 1)
summary_stats_C = summary_stats_C.drop(['comp','club'], axis = 1)

In [168]:
career_stats_total = pd.concat([total_stats,total_stats_B,total_stats_C])

In [169]:
career_stats_summary = pd.concat([summary_stats,summary_stats_B,summary_stats_C])

In [170]:
# Create new database & connection to store career stats

career_sqlite_total_db = '../sql/career_stats_total.db.sqlite'
career_con_total = sqlite3.connect(career_sqlite_total_db) 
career_total = career_con_total.cursor()

# Upload dataframes to SQL database

career_stats_total.to_sql(name = 'total_career_stats', con = career_con_total, if_exists = 'replace', index = False)
career_stats_summary.to_sql(name = 'total_summary_stats', con = career_con_total, if_exists = 'replace', index = False)

### Test accessing data from SQL

In [173]:
sqlite_db = '../sql/career_stats_total.db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

In [174]:
# This is my SQL query to select everything:
table_1 = 'select * from total_career_stats'

# Use `read_sql` from the `pandas` SQL library and set it equal to a DF object:
df_1 = pd.read_sql(table_1, con = conn)

# This is my SQL query to select everything:
table_2 = 'select * from total_summary_stats'

# Use `read_sql` from the `pandas` SQL library and set it equal to a DF object:
df_2 = pd.read_sql(table_2, con = conn)


In [175]:
df_1

,MPG,PPM,apps,assists,clean_sheets,club_id,comp,conceded_goals,goals,league_name,...,pen_goals,pid,pname,red,season,squad,sub_off,sub_on,yel_red,yellow
0,NaN,3.00,1.0,NaN,NaN,7831,A-League Fin.,1.0,NaN,A-League Fin.,...,NaN,39657,michael-theo,NaN,16/17,1.0,1.0,NaN,NaN,NaN
1,NaN,1.43,23.0,NaN,4.0,7831,A-League,32.0,NaN,A-League,...,NaN,39657,michael-theo,1.0,16/17,23.0,NaN,NaN,NaN,3.0
2,NaN,3.00,1.0,NaN,NaN,7831,AFC Champions League,1.0,NaN,AFC Champions League,...,NaN,39657,michael-theo,NaN,2017,2.0,NaN,NaN,NaN,1.0
3,NaN,NaN,NaN,NaN,NaN,7831,A-League Fin.,NaN,NaN,A-League Fin.,...,NaN,39657,michael-theo,NaN,15/16,2.0,NaN,NaN,NaN,NaN
4,NaN,1.78,9.0,NaN,1.0,7831,A-League,20.0,NaN,A-League,...,NaN,39657,michael-theo,NaN,15/16,15.0,NaN,NaN,NaN,2.0
5,NaN,2.60,5.0,NaN,1.0,7831,A-League,7.0,NaN,A-League,...,NaN,39657,michael-theo,NaN,14/15,5.0,NaN,NaN,NaN,NaN
6,NaN,0.00,1.0,NaN,NaN,7831,AFC Champions League,1.0,NaN,AFC Champions League,...,NaN,39657,michael-theo,NaN,2015,2.0,NaN,NaN,NaN,NaN
7,NaN,3.00,2.0,NaN,1.0,7831,A-League Fin.,1.0,NaN,A-League Fin.,...,NaN,39657,michael-theo,NaN,13/14,2.0,NaN,NaN,NaN,NaN
8,NaN,1.84,25.0,NaN,6.0,7831,A-League,24.0,NaN,A-League,...,NaN,39657,michael-theo,NaN,13/14,25.0,NaN,NaN,NaN,3.0
9,NaN,1.50,2.0,NaN,NaN,7831,A-League Fin.,3.0,NaN,A-League Fin.,...,NaN,39657,michael-theo,NaN,12/13,2.0,NaN,NaN,NaN,NaN


In [176]:
df_2

,MPG,PPM,apps,assists,clean_sheets,conceded_goals,goals,intl_caps,intl_country,intl_country_id,...,own_goals,pen_goals,pid,pname,red,squad,sub_off,sub_on,yel_red,yellow
0,NaN,1.65,187.0,NaN,47.0,209.0,NaN,NaN,None,None,...,NaN,NaN,39657,michael-theo,1.0,222.0,2.0,NaN,NaN,15.0
1,NaN,1.28,265.0,NaN,81.0,341.0,NaN,4.0,England U20,22907,...,1.0,NaN,13435,jamie-young,NaN,514.0,5.0,8.0,NaN,8.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,...,NaN,NaN,403669,andre-jannese,NaN,12.0,NaN,NaN,NaN,NaN
3,2736.0,1.32,188.0,NaN,NaN,NaN,6.0,41.0,Australia,3433,...,1.0,NaN,29583,jade-north,NaN,209.0,14.0,4.0,NaN,32.0
4,1323.0,1.36,170.0,NaN,NaN,NaN,11.0,0.0,Australia,3433,...,1.0,NaN,80716,luke-devere,NaN,187.0,17.0,6.0,NaN,12.0
5,2368.0,1.57,90.0,3.0,NaN,NaN,3.0,8.0,Australia U23,32270,...,4.0,NaN,221713,james-donachie,1.0,118.0,3.0,16.0,1.0,11.0
6,1555.0,1.46,83.0,1.0,NaN,NaN,4.0,14.0,Australia U20,22976,...,1.0,NaN,153540,daniel-bowles,1.0,106.0,6.0,15.0,NaN,20.0
7,11151.0,1.52,264.0,11.0,NaN,NaN,2.0,3.0,Australia,3433,...,NaN,NaN,26949,shane-stefanutto,NaN,301.0,31.0,14.0,NaN,35.0
8,3320.0,1.77,84.0,5.0,NaN,NaN,2.0,9.0,Australia U23,32270,...,1.0,NaN,187345,corey-brown,1.0,94.0,4.0,12.0,NaN,21.0
9,579.0,1.56,252.0,17.0,NaN,NaN,33.0,1.0,Germany U20,5709,...,NaN,NaN,10209,jérôme-polenz,2.0,318.0,45.0,36.0,1.0,43.0


### Data cleaning

In [179]:
df_1.head(5)

,MPG,PPM,apps,assists,clean_sheets,club_id,comp,conceded_goals,goals,league_name,...,pen_goals,pid,pname,red,season,squad,sub_off,sub_on,yel_red,yellow
0,NaN,3.00,1.0,NaN,NaN,7831,A-League Fin.,1.0,NaN,A-League Fin.,...,NaN,39657,michael-theo,NaN,16/17,1.0,1.0,NaN,NaN,NaN
1,NaN,1.43,23.0,NaN,4.0,7831,A-League,32.0,NaN,A-League,...,NaN,39657,michael-theo,1.0,16/17,23.0,NaN,NaN,NaN,3.0
2,NaN,3.00,1.0,NaN,NaN,7831,AFC Champions League,1.0,NaN,AFC Champions League,...,NaN,39657,michael-theo,NaN,2017,2.0,NaN,NaN,NaN,1.0
3,NaN,NaN,NaN,NaN,NaN,7831,A-League Fin.,NaN,NaN,A-League Fin.,...,NaN,39657,michael-theo,NaN,15/16,2.0,NaN,NaN,NaN,NaN
4,NaN,1.78,9.0,NaN,1.0,7831,A-League,20.0,NaN,A-League,...,NaN,39657,michael-theo,NaN,15/16,15.0,NaN,NaN,NaN,2.0


In [187]:
# Minutes should be numeric

df_1['minutes'] = df_1['minutes'].apply(pd.to_numeric, errors = 'coerce')
df_2['minutes'] = df_2['minutes'].apply(pd.to_numeric, errors = 'coerce')

,MPG,PPM,apps,assists,clean_sheets,club_id,comp,conceded_goals,goals,league_name,...,pen_goals,pid,pname,red,season,squad,sub_off,sub_on,yel_red,yellow


In [200]:
# International country ID should be numeric

df_2['intl_country_id'] = df_2['intl_country_id'].apply(pd.to_numeric, errors = 'coerce')

In [207]:
# Review leagues where the season is expressed as a calendar year instead of over two years (e.g. 2017 vs 16/17)

df_1[df_1['season'] == "2016"]['league_name'].unique()

array([u'AFC Champions League', u'NM-Cup', u'Tippeligaen',
       u'J1 - 2nd Stage', u"Emperor's Cup", u'J1 - 1st Stage',
       u'J. League Cup', u'ISL Play-Offs', u'ISL', u'K League Classic',
       u'AFC CL-Qualifikation', u'Super League', u'Allsvenskan',
       u'Thai League', u'Relegation Eliteserien', u'OBOS-ligaen',
       u'Liigacup', u'Premier Liga', u'S\xe9rie A', u'Libertadores',
       u'United Football League', u'Primera Divisi\xf3n', u'Veikkausliiga',
       u'Suomen Cup', u'A Lyga', u'Chinese FA Cup', u'Chinese Super Cup',
       u'MLS', u'USL', u'Copa Sudamericana', u'League One',
       u'NASL Fall Championship', u'US Open Cup',
       u'NASL Spring Championship', u'Liga \xc1guila I',
       u'CAF-Champions League', u'Relegation Allsvenskan', u'Superettan',
       u'MLS Cup Playoffs', u'USL Playoffs', u'NASL Playoffs',
       u'S\xe9rie B', u'Canadian Championship', u'Torneo Clausura',
       u'Descentralizado Playoffs', u'Liguilla B', u'Liga \xc1guila II',
       u'U2

In [216]:
df_1['season'].unique()
df_1['season'] = df_1['season'].map(lambda x: np.NaN if x == 'None' else x)

In [217]:
df_1['season'].unique()

array([u'16/17', u'2017', u'15/16', u'14/15', u'2015', u'13/14', u'12/13',
       u'2013', u'2012', u'11/12', u'10/11', u'09/10', u'08/09', u'07/08',
       u'06/07', u'05/06', u'02/03', u'04/05', u'03/04', u'2011', u'2010',
       u'2014', u'2009', u'2008', u'2007', u'2006', u'2005', u'2004',
       u'01/02', u'00/01', u'17/18', u'2016', u'99/00', u'95/96', u'98/99',
       u'97/98', u'96/97', u'94/95', u'80/81', u'2003', u'2002', u'2001',
       u'2000', u'1999', u'2018', u'93/94', u'92/93', None], dtype=object)

In [283]:
# Create new column in df_1 to convert all season numbers to same expression, whereby 2017 = 2017 and 16/17 = 2016

def season_to_year(value):
    
    if value is not None:
        
        if len(value) == 4: # If statement for where season is four digit e.g. 2017
            make_float = float(value)
            year = make_float

        elif len(value) == 5: # If statement for where season is 5 characters e.g. 16/17
            first_two_char = value[0:2]
            if value[0] == "9" or value[0] == "8":
                year = float("19" + first_two_char)
            else:
                year = float("20" + first_two_char)
    
    else:
        year = np.NaN
    
    return year

df_1['year'] = df_1['season'].apply(season_to_year)

In [236]:
df_2.head()

,MPG,PPM,apps,assists,clean_sheets,conceded_goals,goals,intl_caps,intl_country,intl_country_id,...,own_goals,pen_goals,pid,pname,red,squad,sub_off,sub_on,yel_red,yellow
0,NaN,1.65,187.0,NaN,47.0,209.0,NaN,NaN,None,NaN,...,NaN,NaN,39657,michael-theo,1.0,222.0,2.0,NaN,NaN,15.0
1,NaN,1.28,265.0,NaN,81.0,341.0,NaN,4.0,England U20,22907.0,...,1.0,NaN,13435,jamie-young,NaN,514.0,5.0,8.0,NaN,8.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,...,NaN,NaN,403669,andre-jannese,NaN,12.0,NaN,NaN,NaN,NaN
3,2736.0,1.32,188.0,NaN,NaN,NaN,6.0,41.0,Australia,3433.0,...,1.0,NaN,29583,jade-north,NaN,209.0,14.0,4.0,NaN,32.0
4,1323.0,1.36,170.0,NaN,NaN,NaN,11.0,0.0,Australia,3433.0,...,1.0,NaN,80716,luke-devere,NaN,187.0,17.0,6.0,NaN,12.0


## Save dataframes to CSV

In [287]:
start_time=time.time()
start_time_stamp = str(start_time).replace(".","")
start_time_stamp

url_content_file_name = "career_by_year_" + start_time_stamp + ".csv"

df_1.to_csv(url_content_file_name, encoding="utf-8", index = False)

In [239]:
start_time=time.time()
start_time_stamp = str(start_time).replace(".","")
start_time_stamp

url_content_file_name = "career_summary_" + start_time_stamp + ".csv"

df_2.to_csv(url_content_file_name, encoding="utf-8", index = False)

## Load other SQL databases and convert to CSV

#### Market value dataframe

In [241]:
marketvals_db = '../sql/stats.db.sqlite'
conn_2 = sqlite3.connect(marketvals_db) 
c = conn.cursor()

# This is my SQL query to select everything:
table_3 = 'select * from market_vals'

# Use `read_sql` from the `pandas` SQL library and set it equal to a DF object:
df_3 = pd.read_sql(table_3, con = conn_2)

In [243]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16530 entries, 0 to 16529
Data columns (total 26 columns):
teamid            16530 non-null float64
number            12371 non-null float64
name              16530 non-null object
pid               16530 non-null float64
main_pos          16530 non-null object
position          16530 non-null object
dob               16507 non-null object
age               16530 non-null float64
nat1              16530 non-null object
nat2              5103 non-null object
height            15909 non-null float64
foot              14835 non-null object
teamsince         14884 non-null object
prevteam          14884 non-null object
prevteamid        14523 non-null float64
transferfee       16530 non-null float64
contractuntil     12791 non-null object
marketval         14919 non-null float64
prevmarketval     14367 non-null float64
team              16530 non-null object
season            16530 non-null object
teamurl           16530 non-null object
yea

In [246]:
# Convert date fields to datetime

date_fields = ['dob','teamsince','contractuntil']

for col in date_fields:
    df_3[col] = pd.to_datetime(df_3[col], format="%Y-%m-%d")

In [280]:
# Check matching of season to years

temp_df_3 = df_3[['season','year']].drop_duplicates()
temp_df_3

,season,year
0,15/16,2015.0
301,16/17,2016.0
593,17/18,2017.0
2940,2016,2015.0
3581,2017,2016.0


In [285]:
# Change year to match season where year is 2016 or 2017 (since these are calendar years for MLS)
df_3['year'] = df_3['season'].apply(season_to_year)

# Re-check matching of season to years

temp_df_3 = df_3[['season','year']].drop_duplicates()
temp_df_3

,season,year
0,15/16,2015.0
301,16/17,2016.0
593,17/18,2017.0
2940,2016,2016.0
3581,2017,2017.0


In [257]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16530 entries, 0 to 16529
Data columns (total 26 columns):
teamid            16530 non-null float64
number            12371 non-null float64
name              16530 non-null object
pid               16530 non-null float64
main_pos          16530 non-null object
position          16530 non-null object
dob               16507 non-null datetime64[ns]
age               16530 non-null float64
nat1              16530 non-null object
nat2              5103 non-null object
height            15909 non-null float64
foot              14835 non-null object
teamsince         14884 non-null datetime64[ns]
prevteam          14884 non-null object
prevteamid        14523 non-null float64
transferfee       16530 non-null float64
contractuntil     12791 non-null datetime64[ns]
marketval         14919 non-null float64
prevmarketval     14367 non-null float64
team              16530 non-null object
season            16530 non-null object
teamurl           1

In [286]:
# Convert dataframe to CSV

start_time=time.time()
start_time_stamp = str(start_time).replace(".","")
start_time_stamp

url_content_file_name = "market_values_" + start_time_stamp + ".csv"

df_3.to_csv(url_content_file_name, encoding="utf-8", index = False)

#### Player stats (from Squawka dataframes)

In [260]:
player_stats_db = '../sql/player_stats.db.sqlite'
conn_3 = sqlite3.connect(player_stats_db) 
c = conn.cursor()

# This is my SQL query to select everything:
table_4 = 'select * from playertable'

# Use `read_sql` from the `pandas` SQL library and set it equal to a DF object:
df_4 = pd.read_sql(table_4, con = conn_3)

In [261]:
df_4

,age,bmi,country,dob,first_name,height,last_name,name,position,shirt_num,state,surname,team_id,team_name,weight,idn
0,27.0,21.2,Wales,1989-02-07,Neil,175.0,Taylor,Neil Taylor,Defender,0.0,playing,Neil Taylor,46.0,Swansea,65.0,817.0
1,32.0,23.0,Wales,1984-08-23,Ashley,183.0,Williams,Ashley Williams,Defender,0.0,playing,Ashley Williams,46.0,Swansea,77.0,810.0
2,25.0,22.2,England,1991-05-22,Kyle,185.0,Bartley,Kyle Bartley,Defender,0.0,bench,Kyle Bartley,46.0,Swansea,76.0,382.0
3,31.0,23.0,Poland,1985-04-18,Lukasz,190.0,Fabianski,Lukasz Fabianski,Goalkeeper,1.0,playing,Fabianski,46.0,Swansea,83.0,375.0
4,28.0,22.6,Korea Republic,1989-01-24,Sung-yueng,187.0,Ki,Ki Sung-yueng,Midfielder,4.0,playing,Ki,46.0,Swansea,79.0,3691.0
5,28.0,20.4,Netherlands,1988-05-27,Marvin,180.0,Emnes,Marvin Emnes,Forward,11.0,bench,Marvin Emnes,46.0,Swansea,66.0,10771.0
6,24.0,23.4,England,1992-02-27,Jonjo,185.0,Shelvey,Jonjo Shelvey,Midfielder,12.0,playing,Shelvey,46.0,Swansea,80.0,523.0
7,29.0,22.0,England,1987-11-29,Nathan,165.0,Dyer,Nathan Dyer,Midfielder,12.0,bench,Nathan Dyer,46.0,Swansea,60.0,812.0
8,28.0,25.4,Côte d'Ivoire,1988-12-10,Wilfried,182.0,Bony,Wilfried Bony,Forward,14.0,playing,Bony,46.0,Swansea,84.0,5574.0
9,32.0,22.1,England,1985-01-07,Wayne,170.0,Routledge,Wayne Routledge,Midfielder,15.0,playing,Routledge,46.0,Swansea,64.0,826.0


In [262]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292299 entries, 0 to 292298
Data columns (total 16 columns):
age           292299 non-null float64
bmi           292299 non-null float64
country       281805 non-null object
dob           292299 non-null object
first_name    292299 non-null object
height        283778 non-null float64
last_name     292299 non-null object
name          292299 non-null object
position      292299 non-null object
shirt_num     292299 non-null float64
state         292299 non-null object
surname       292299 non-null object
team_id       292299 non-null float64
team_name     292299 non-null object
weight        281595 non-null float64
idn           292299 non-null float64
dtypes: float64(7), object(9)
memory usage: 35.7+ MB


In [267]:
df_4['dob'] = df_4['dob'].map(lambda x: np.NaN if x == "0000-00-00" else x)
df_4['dob'] = df_4['dob'].map(lambda x: "1996-01-22" if x == "1996-00-22" else x) # To fix an error for one player
df_4['dob'] = pd.to_datetime(df_4['dob'], format="%Y-%m-%d")

In [268]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292299 entries, 0 to 292298
Data columns (total 16 columns):
age           292299 non-null float64
bmi           292299 non-null float64
country       281805 non-null object
dob           292082 non-null datetime64[ns]
first_name    292299 non-null object
height        283778 non-null float64
last_name     292299 non-null object
name          292299 non-null object
position      292299 non-null object
shirt_num     292299 non-null float64
state         292299 non-null object
surname       292299 non-null object
team_id       292299 non-null float64
team_name     292299 non-null object
weight        281595 non-null float64
idn           292299 non-null float64
dtypes: datetime64[ns](1), float64(7), object(8)
memory usage: 35.7+ MB


In [252]:
# This is my SQL query to select everything:
table_5 = 'select * from stats'

# Use `read_sql` from the `pandas` SQL library and set it equal to a DF object:
df_5 = pd.read_sql(table_5, con = conn_3)

In [269]:
df_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221081 entries, 0 to 221080
Data columns (total 17 columns):
idn            221081 non-null float64
attack         221081 non-null float64
defense        221081 non-null float64
possession     221081 non-null float64
goalkeeping    221081 non-null float64
score          221081 non-null float64
match_id       221081 non-null object
start_year     221081 non-null int64
season_code    221081 non-null object
goals          221081 non-null int64
assists        221081 non-null int64
team_id        221081 non-null float64
state          221081 non-null int64
age            221081 non-null float64
sub            221081 non-null int64
result         221081 non-null object
clean_sheet    221081 non-null int64
dtypes: float64(8), int64(6), object(3)
memory usage: 28.7+ MB


In [272]:
#Convert match id to numeric 
df_5['match_id'] = df_5['match_id'].apply(pd.to_numeric, errors = 'coerce')

In [273]:
df_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221081 entries, 0 to 221080
Data columns (total 17 columns):
idn            221081 non-null float64
attack         221081 non-null float64
defense        221081 non-null float64
possession     221081 non-null float64
goalkeeping    221081 non-null float64
score          221081 non-null float64
match_id       221081 non-null int64
start_year     221081 non-null int64
season_code    221081 non-null object
goals          221081 non-null int64
assists        221081 non-null int64
team_id        221081 non-null float64
state          221081 non-null int64
age            221081 non-null float64
sub            221081 non-null int64
result         221081 non-null object
clean_sheet    221081 non-null int64
dtypes: float64(8), int64(7), object(2)
memory usage: 28.7+ MB


In [281]:
df_5.head()

,idn,attack,defense,possession,goalkeeping,score,match_id,start_year,season_code,goals,assists,team_id,state,age,sub,result,clean_sheet
0,67.0,0.00,-27.30,-1.43,0.0,-28.73,8445,2014,14/15,0,0,315.0,1,30.0,0,L,0
1,375.0,0.00,55.21,4.42,0.0,59.63,8445,2014,14/15,0,0,46.0,1,31.0,0,W,1
2,523.0,10.19,-8.28,-7.88,0.0,-5.97,8445,2014,14/15,0,0,46.0,1,24.0,0,W,1
3,728.0,0.00,14.45,17.12,0.0,31.57,8445,2014,14/15,0,0,315.0,1,33.0,0,L,0
4,807.0,0.00,-2.04,7.05,0.0,5.01,8445,2014,14/15,0,0,46.0,1,34.0,0,W,1


In [282]:
# Check matching of season to years

temp_df_5 = df_5[['start_year','season_code']].drop_duplicates()
temp_df_5

,start_year,season_code
0,2014,14/15
9610,2015,15/16
88486,2012,12/13
92854,2013,13/14
112170,2016,16/17


#### Access additional player stats scraped for player details per teams they played in 2017/18 season

In [288]:
extra_player_stats_db = '../sql/stats_extra_games.db.sqlite'
conn_6 = sqlite3.connect(extra_player_stats_db) 
c = conn.cursor()

# This is my SQL query to select everything:
table_6 = 'select * from playertable'

# Use `read_sql` from the `pandas` SQL library and set it equal to a DF object:
df_6 = pd.read_sql(table_6, con = conn_6)

In [289]:
df_6

,age,bmi,country,dob,first_name,height,last_name,name,position,shirt_num,state,surname,team_id,team_name,weight,idn
0,24.0,24.7,England,1993-03-10,Jack,196.0,Butland,Jack Butland,Goalkeeper,1.0,playing,Butland,44.0,Stoke,95.0,92.0
1,29.0,25.1,Netherlands,1988-08-07,Erik,183.0,Pieters,Erik Pieters,Defender,3.0,playing,Pieters,44.0,Stoke,84.0,5429.0
2,27.0,22.0,Wales,1990-03-14,Joe,168.0,Allen,Joe Allen,Midfielder,4.0,playing,Joe Allen,44.0,Stoke,62.0,540.0
3,23.0,26.6,France,1994-10-27,Kurt,190.0,Zouma,Kurt Zouma,Defender,6.0,playing,Kurt Zouma,44.0,Stoke,96.0,1419.0
4,33.0,23.5,England,1984-08-23,Glen,182.0,Johnson,Glen Johnson,Defender,8.0,bench,Glen Johnson,44.0,Stoke,78.0,75.0
5,24.0,25.3,England,1993-08-04,Saido,180.0,Berahino,Saido Berahino,Forward,9.0,bench,Berahino,44.0,Stoke,82.0,886.0
6,28.0,24.9,Cameroon,1989-03-23,Eric Maxim,191.0,Choupo-Moting,Eric Maxim Choupo-Moting,Midfielder,10.0,playing,Choupo-Moting,44.0,Stoke,91.0,2000.0
7,24.0,22.7,Spain,1993-02-26,Jesé,178.0,Rodríguez Ruiz,Jesé,Forward,11.0,playing,Rodríguez Ruiz,44.0,Stoke,72.0,7109.0
8,26.0,22.2,Netherlands,1992-02-08,Bruno,185.0,Martins Indi,Bruno Martins Indi,Defender,15.0,playing,Martins Indi,44.0,Stoke,76.0,7818.0
9,32.0,24.3,Scotland,1985-12-10,Charlie,185.0,Adam,Charlie Adam,Midfielder,16.0,bench,Adam,44.0,Stoke,83.0,531.0


In [290]:
df_6['dob'] = df_6['dob'].map(lambda x: np.NaN if x == "0000-00-00" else x)
df_6['dob'] = df_6['dob'].map(lambda x: "1996-01-22" if x == "1996-00-22" else x) # To fix an error for one player
df_6['dob'] = pd.to_datetime(df_6['dob'], format="%Y-%m-%d")

In [291]:
# This is my SQL query to select everything:
table_7 = 'select * from stats'

# Use `read_sql` from the `pandas` SQL library and set it equal to a DF object:
df_7 = pd.read_sql(table_7, con = conn_6)

In [292]:
#Convert match id to numeric 
df_7['match_id'] = df_7['match_id'].apply(pd.to_numeric, errors = 'coerce')

In [293]:
temp_df_7 = df_7[['start_year','season_code']].drop_duplicates()
temp_df_7

,start_year,season_code
0,2017,17/18


#### Append additional stats to existing dataframes

In [294]:
df_4 = pd.concat([df_4,df_6])
df_4 = df_4.reset_index()

df_5 = pd.concat([df_5,df_7])
df_5 = df_5.reset_index()

In [295]:
df_4.shape

(297116, 17)

In [296]:
df_5.shape

(224576, 18)

#### Convert Squawka dataframes to CSV

In [297]:
# Convert dataframes to CSV

start_time=time.time()
start_time_stamp = str(start_time).replace(".","")
start_time_stamp

player_info_file_name = "player_info_" + start_time_stamp + ".csv"
player_stats_file_name = "player_stats_" + start_time_stamp + ".csv"

df_4.to_csv(player_info_file_name, encoding="utf-8", index = False)
df_5.to_csv(player_stats_file_name, encoding="utf-8", index = False)